# Analysis of short 20 sec Power Spectra

In [2]:

import plotly
import plotly.graph_objs as go

# Importing Python and external packages


import os
import sys
import importlib
from importlib import reload 
from dataclasses import dataclass, field, fields
from itertools import compress
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler

import scipy
from scipy import stats
from scipy import signal
from scipy.signal import spectrogram, hann, butter, filtfilt, freqz

import seaborn as sns
import pingouin as pg
from itertools import combinations
from statannotations.Annotator import Annotator

import openpyxl
from openpyxl import Workbook, load_workbook
# import xlrd
import pickle
import json
import csv
import h5py
import fooof
from fooof.plts.spectra import plot_spectrum

#mne
from pathlib import Path
import mne_bids
from mne_bids import (
    BIDSPath,
    inspect_dataset,
    mark_channels)
import mne
from mne.time_frequency import tfr_morlet 

/Users/jenniferbehnke/anaconda3/envs/bssu/lib/python3.11/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [100]:
jennifer_user_path = os.getcwd()
while jennifer_user_path[-14:] != 'jenniferbehnke':
    jennifer_user_path = os.path.dirname(jennifer_user_path)

# directory to this Repository
project_path = os.path.join(jennifer_user_path, 'code', 'Monopolar_power_estimation', 'monopolar_directional_beta')
sys.path.append(project_path)

os.chdir(project_path)

import src.monopolar_bssu.utils.find_folders as find_folders
importlib.reload(find_folders)

# import PyPerceive

project_path = find_folders.chdir_repository("Py_Perceive")

from PerceiveImport.classes import (
    main_class, modality_class, metadata_class,
    session_class, condition_class, task_class,
    contact_class, run_class
)

import PerceiveImport.methods.load_rawfile as load_rawfile
import PerceiveImport.methods.find_folders as PyPerceive_find_folders
import PerceiveImport.methods.metadata_helpers as metaHelpers

# import meet

project_path = find_folders.chdir_repository("meet")

import meet as meet

# import all functions from BetaSenSightLongterm
project_path = find_folders.chdir_repository("monopolar_directional_beta")


# externalized LFP
import src.monopolar_bssu.externalized_lfp.externalized_lfp as externalized
import src.monopolar_bssu.externalized_lfp.feats_ssd as feats_ssd

# monopolar Referencing
import src.monopolar_bssu.monopol_bssu_methods.MonoRef_JLB as MonoRefJLB
#import src.bssu.monopolar.GroupMonopolarPSD as groupMonopol
import src.monopolar_bssu.monopol_bssu_methods.monoRef_weightPsdAverageByCoordinateDistance as MonoRefWeightedCoordinateDistance
import src.monopolar_bssu.monopol_bssu_methods.bssu_contacts_maximal_beta as bssu_contacts
import src.monopolar_bssu.monopol_bssu_methods.monoRef_Strelow as detec_strelow

# Power Spectra
import src.monopolar_bssu.power_spectra.power_spectra_externalized as power_spectra_externalized

# Clinical stimulation parameters
import src.monopolar_bssu.clinical_contacts.activeStimulationContacts as activeStimContacts

# comparison analysis
import src.monopolar_bssu.comparison_analysis.monopol_comparison_tests as monopol_comparison_tests
import src.monopolar_bssu.comparison_analysis.monopol_method_comparison as monopol_method_comparison

# utility functions
import src.monopolar_bssu.utils.loadResults as loadResults
import src.monopolar_bssu.utils.find_folders as find_folders
import src.monopolar_bssu.utils.io_externalized as io_externalized
import src.monopolar_bssu.utils.io_percept as io_percept
import src.monopolar_bssu.utils.io_monopolar_comparison as io_monopolar_comparison
import src.monopolar_bssu.utils.externalized_lfp_preprocessing as externalized_lfp_preprocessing
import src.monopolar_bssu.utils.percept_lfp_preprocessing as percept_lfp_preprocessing
import src.monopolar_bssu.utils.tmsi_poly5reader as tmsi_poly5reader

# short time stability 
import src.monopolar_bssu.short_time_stability_power.externalized_short_chunks as externalized_short_chunks


importlib.reload(externalized)
importlib.reload(feats_ssd)
importlib.reload(loadResults)
importlib.reload(MonoRefJLB)
importlib.reload(MonoRefWeightedCoordinateDistance)
importlib.reload(bssu_contacts)
importlib.reload(detec_strelow)
importlib.reload(power_spectra_externalized)
importlib.reload(activeStimContacts)
importlib.reload(monopol_comparison_tests)
importlib.reload(monopol_method_comparison)
importlib.reload(loadResults)
importlib.reload(find_folders)
importlib.reload(io_externalized)
importlib.reload(io_percept)
importlib.reload(io_monopolar_comparison)
importlib.reload(externalized_lfp_preprocessing)
importlib.reload(percept_lfp_preprocessing)
importlib.reload(tmsi_poly5reader)
importlib.reload(externalized_short_chunks)


# load if you want to see complete Dataframes
pd.set_option("display.max_rows", None)

Excel file loaded:  patient_metadata.xlsx 
loaded from:  /Users/jenniferbehnke/Dropbox/work/ResearchProjects/Monopolar_power_estimation/data
Excel file loaded:  patient_metadata.xlsx 
loaded from:  /Users/jenniferbehnke/Dropbox/work/ResearchProjects/Monopolar_power_estimation/data
Excel file loaded:  patient_metadata.xlsx 
loaded from:  /Users/jenniferbehnke/Dropbox/work/ResearchProjects/Monopolar_power_estimation/data


In [3]:
bssu_data_loaded = io_externalized.load_externalized_pickle(filename="externalized_directional_bssu_channels", reference="bipolar_to_lowermost")

In [4]:
bssu_data_loaded.loc[bssu_data_loaded.BIDS_id == "noBIDS24"].head()

,BIDS_id,subject,hemisphere,session,subject_hemisphere,bipolar_channel,time_stamps_250Hz,lfp_resampled_250Hz,filtered_lfp_250Hz,only_high_pass_lfp_250Hz,n_samples_250Hz
0,noBIDS24,024,Right,pre-IPG,024_Right,01,"[0.0, 0.004, 0.008, 0.012, 0.016, 0.02, 0.024,...","[-0.05402010422003183, -0.05403081637623904, -...","[5.2580631866917335e-08, -8.07395841279414e-06...","[2.5097978689854487e-06, -7.857326029623908e-0...",30000
0,noBIDS24,024,Right,pre-IPG,024_Right,02,"[0.0, 0.004, 0.008, 0.012, 0.016, 0.02, 0.024,...","[-0.056680708938918, -0.05668959381403876, -0....","[4.051196847962224e-07, -5.867921586363009e-06...","[1.5092891581170489e-06, -7.141439125523177e-0...",30000
0,noBIDS24,024,Right,pre-IPG,024_Right,03,"[0.0, 0.004, 0.008, 0.012, 0.016, 0.02, 0.024,...","[-0.010260039408472432, -0.010269640405165523,...","[3.3754137369927164e-07, -7.440385359100413e-0...","[2.0044483669667603e-06, -7.321759874859456e-0...",30000
0,noBIDS24,024,Right,pre-IPG,024_Right,12,"[0.0, 0.004, 0.008, 0.012, 0.016, 0.02, 0.024,...","[-0.00266060471888617, -0.002658777437799721, ...","[3.525390530730823e-07, 2.2060368266151104e-06...","[-1.0006768531720542e-06, 7.157169106543659e-0...",30000
0,noBIDS24,024,Right,pre-IPG,024_Right,13,"[0.0, 0.004, 0.008, 0.012, 0.016, 0.02, 0.024,...","[0.043760064811559404, 0.04376117597107352, 0....","[2.849607418726738e-07, 6.335730536778569e-07,...","[-5.054836602006715e-07, 5.354331240126414e-07...",30000


## Load the externalized Power Spectra 2 min and 20 sec chunks: 2 filter options
- notch_and_band_pass_filtered: to analyze beta power without FOOOF
- only_high_pass_filtered: to run FOOOF on 20sec chunks and then analyze periodic beta power

1. To write 20 sec power spectra chunks: use externalized_lfp.ipynb and power_spectra_externalized.py 

2. Write group dataframe 

In [14]:
group_chunk_df = power_spectra_externalized.group_20sec_power_spectra_externalized_bssu(incl_bids_id=["all"], filtered="notch_and_band_pass_filtered")


No data for L002
power_spectra_BSSU_externalized_20sec_group_notch_and_band_pass_filtered.pickle 
written in: /Users/jenniferbehnke/Dropbox/work/ResearchProjects/Monopolar_power_estimation/results


In [13]:
load_group_df = io_externalized.load_externalized_pickle(filename="power_spectra_BSSU_externalized_20sec_group_notch_and_band_pass_filtered")
load_group_df.head()

,bids_id,hemisphere,channel,chunks,fourier_transform_2min,filtered,subject
0,noBIDS24,Right,01,"{1: [5.2580631866917335e-08, -8.07395841279414...","{'frequencies': [0.0, 1.0, 2.0, 3.0, 4.0, 5.0,...",notch_and_band_pass_filtered,024
1,noBIDS24,Right,02,"{1: [4.051196847962224e-07, -5.867921586363009...","{'frequencies': [0.0, 1.0, 2.0, 3.0, 4.0, 5.0,...",notch_and_band_pass_filtered,024
2,noBIDS24,Right,03,"{1: [3.3754137369927164e-07, -7.44038535910041...","{'frequencies': [0.0, 1.0, 2.0, 3.0, 4.0, 5.0,...",notch_and_band_pass_filtered,024
3,noBIDS24,Right,12,"{1: [3.525390530730823e-07, 2.2060368266151104...","{'frequencies': [0.0, 1.0, 2.0, 3.0, 4.0, 5.0,...",notch_and_band_pass_filtered,024
4,noBIDS24,Right,13,"{1: [2.849607418726738e-07, 6.335730536778569e...","{'frequencies': [0.0, 1.0, 2.0, 3.0, 4.0, 5.0,...",notch_and_band_pass_filtered,024


## NON-FOOOF band-pass and notch-filtered Power Spectra analysis

Reorganize Dataframe 
- columns: sub_hem, channel, power_spectrum_20sec_1, power_spectrum_20sec_2, power_spectrum_20sec_3, power_spectrum_20sec_4, power_spectrum_2min


In [37]:
all_power_spectra = externalized_short_chunks.reorganize_data(filtered="notch_and_band_pass_filtered")
all_power_spectra.head()

20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist


,subject,hemisphere,channel,power_spectrum_2min,frequencies,power_spectrum_20sec_1,power_spectrum_20sec_2,power_spectrum_20sec_3,power_spectrum_20sec_4,power_spectrum_20sec_5
0,024,Right,01,"[4.243743386290976e-15, 1.9523421136845138e-15...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[4.4144655014073264e-15, 1.9754081948192734e-1...","[5.4575673240417846e-15, 2.522135666936564e-15...","[3.3914109102844663e-15, 1.5442941700951845e-1...","[3.8131056939255006e-15, 1.8326184402692994e-1...","[3.8769148226844085e-15, 1.8162783229698833e-1..."
1,024,Right,02,"[5.8113147039513626e-15, 2.6794372982506884e-1...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[4.845491280874276e-15, 2.1871545403575145e-15...","[7.197703979956034e-15, 3.274044547371873e-15,...","[5.069692514010366e-15, 2.2982011765744405e-15...","[4.5021524947930285e-15, 2.165131435269984e-15...","[5.404487511144461e-15, 2.5094920910978397e-15..."
2,024,Right,03,"[5.797080343303016e-15, 2.663030265729246e-15,...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[5.106539743681243e-15, 2.328071732519717e-15,...","[7.932077877413296e-15, 3.6072978470978506e-15...","[5.236638094522156e-15, 2.3639022028672775e-15...","[4.186957399372503e-15, 1.9859279655801783e-15...","[5.78628318355391e-15, 2.672205556678135e-15, ..."
3,024,Right,12,"[6.542958870908336e-16, 2.9357164970206494e-16...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[5.92287045607634e-16, 2.594201294091107e-16, ...","[7.877200118839062e-16, 3.5006838499647934e-16...","[5.031311869087732e-16, 2.197451140536463e-16,...","[5.58958100327173e-16, 2.6140143342955563e-16,...","[7.266723126549889e-16, 3.321520753945198e-16,..."
4,024,Right,13,"[1.2610617066472904e-15, 5.616117728814826e-16...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[1.118106912570284e-15, 4.913744010980446e-16,...","[1.5831856526548015e-15, 7.0373430855823e-16, ...","[1.1886839611985835e-15, 5.173736899634197e-16...","[1.4174786620478432e-15, 6.586696129620791e-16...","[1.3087989921764397e-15, 5.780030834626737e-16..."


Get beta, high beta, low beta mean ± SD

In [38]:
power_spectrum = all_power_spectra.power_spectrum_2min.values[0]
frequencies = all_power_spectra.frequencies.values[0]

In [42]:
frequency_bands = externalized_short_chunks.caculate_freq_band_mean_sd(power_spectrum, frequencies)
frequency_bands

{'beta_power': 5.878167641710751e-13,
 'high_beta_power': 5.192550867991468e-13,
 'low_beta_power': 7.163699092434409e-13,
 'beta_power_sd': 3.3764791297369715e-13,
 'high_beta_power_sd': 3.9139835903560926e-13,
 'low_beta_power_sd': 1.232541213079061e-13,
 'beta_power_freq': array([13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
        26., 27., 28., 29., 30., 31., 32., 33., 34., 35.]),
 'high_beta_power_freq': array([21., 22., 23., 24., 25., 26., 27., 28., 29., 30., 31., 32., 33.,
        34., 35.]),
 'low_beta_power_freq': array([13., 14., 15., 16., 17., 18., 19., 20.])}

In [50]:
freq_band_results = externalized_short_chunks.frequency_band_mean_sd(filtered="notch_and_band_pass_filtered")

20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist


In [52]:
freq_band_results["beta_band_result"].head()

,subject,hemisphere,channel,frequencies,power_spectrum_2min,power_spectrum_20sec_1,power_spectrum_20sec_2,power_spectrum_20sec_3,power_spectrum_20sec_4,power_spectrum_20sec_5,...,power_spectrum_20sec_2_mean,power_spectrum_20sec_3_mean,power_spectrum_20sec_4_mean,power_spectrum_20sec_5_mean,power_spectrum_2min_sd,power_spectrum_20sec_1_sd,power_spectrum_20sec_2_sd,power_spectrum_20sec_3_sd,power_spectrum_20sec_4_sd,power_spectrum_20sec_5_sd
0,024,Right,01,"[13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20....","[1.006959873681143e-12, 7.627618606373016e-13,...","[9.227261489012247e-13, 6.856116072561524e-13,...","[1.6238086629250652e-12, 1.0183756423828129e-1...","[7.63183402082647e-13, 6.122674865474809e-13, ...","[7.333007422693277e-13, 6.482917709124258e-13,...","[1.0377924563977723e-12, 8.049693773248429e-13...",...,6.593876e-13,5.230051e-13,6.110900e-13,5.399850e-13,3.376479e-13,4.029409e-13,4.310370e-13,2.884047e-13,4.183058e-13,3.434652e-13
1,024,Right,02,"[13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20....","[1.160899811117978e-12, 8.567658832017647e-13,...","[1.112888572779954e-12, 8.456427848714715e-13,...","[1.9864805111607753e-12, 1.22709808887544e-12,...","[1.0017094867206536e-12, 7.040899407980877e-13...","[8.091607529056341e-13, 7.076689567011787e-13,...","[1.3069238039228217e-12, 9.088034864031947e-13...",...,7.042918e-13,5.722276e-13,6.223233e-13,5.703442e-13,3.570969e-13,4.162824e-13,4.730358e-13,3.096919e-13,4.176669e-13,3.454101e-13
2,024,Right,03,"[13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20....","[9.511951793724103e-13, 7.529656792317256e-13,...","[8.999942123011992e-13, 7.426553074306858e-13,...","[1.6925390976709618e-12, 1.0601763124280895e-1...","[7.844179708194547e-13, 5.957220911469509e-13,...","[6.283115748756354e-13, 6.552149821402667e-13,...","[1.0470446388290749e-12, 7.480233255728902e-13...",...,6.305493e-13,5.153313e-13,5.580002e-13,5.031370e-13,3.149557e-13,3.711278e-13,4.069026e-13,2.667660e-13,3.802697e-13,2.877744e-13
3,024,Right,12,"[13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20....","[9.441147341234419e-14, 7.274418241094575e-14,...","[9.121987733213792e-14, 8.119072538923668e-14,...","[1.267965799606627e-13, 8.919558150812431e-14,...","[7.063571409934557e-14, 5.611350518905433e-14,...","[8.909305949369696e-14, 5.817558001771653e-14,...","[9.618901671729255e-14, 6.49955013827536e-14, ...",...,5.152335e-14,4.326514e-14,4.518448e-14,4.460339e-14,2.015693e-14,2.128565e-14,3.032828e-14,1.755182e-14,2.147577e-14,2.214879e-14
4,024,Right,13,"[13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20....","[1.4216056694099896e-13, 1.1115620195516617e-1...","[9.671092120251205e-14, 1.0093164860146917e-13...","[1.829484338946015e-13, 1.2513300293004138e-13...","[8.887085101658428e-14, 8.8573188110667e-14, 9...","[1.5964162842363502e-13, 1.0326965008732892e-1...","[1.397320875890213e-13, 1.1576942786223027e-13...",...,8.291981e-14,6.933578e-14,7.448761e-14,6.836462e-14,3.173297e-14,2.538796e-14,4.437440e-14,2.578828e-14,4.175718e-14,3.186879e-14


In [59]:
beta = freq_band_results["beta_band_result"]
beta.head()



,subject,hemisphere,channel,frequencies,power_spectrum_2min,power_spectrum_20sec_1,power_spectrum_20sec_2,power_spectrum_20sec_3,power_spectrum_20sec_4,power_spectrum_20sec_5,...,power_spectrum_20sec_2_mean,power_spectrum_20sec_3_mean,power_spectrum_20sec_4_mean,power_spectrum_20sec_5_mean,power_spectrum_2min_sd,power_spectrum_20sec_1_sd,power_spectrum_20sec_2_sd,power_spectrum_20sec_3_sd,power_spectrum_20sec_4_sd,power_spectrum_20sec_5_sd
0,024,Right,01,"[13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20....","[1.006959873681143e-12, 7.627618606373016e-13,...","[9.227261489012247e-13, 6.856116072561524e-13,...","[1.6238086629250652e-12, 1.0183756423828129e-1...","[7.63183402082647e-13, 6.122674865474809e-13, ...","[7.333007422693277e-13, 6.482917709124258e-13,...","[1.0377924563977723e-12, 8.049693773248429e-13...",...,6.593876e-13,5.230051e-13,6.110900e-13,5.399850e-13,3.376479e-13,4.029409e-13,4.310370e-13,2.884047e-13,4.183058e-13,3.434652e-13
1,024,Right,02,"[13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20....","[1.160899811117978e-12, 8.567658832017647e-13,...","[1.112888572779954e-12, 8.456427848714715e-13,...","[1.9864805111607753e-12, 1.22709808887544e-12,...","[1.0017094867206536e-12, 7.040899407980877e-13...","[8.091607529056341e-13, 7.076689567011787e-13,...","[1.3069238039228217e-12, 9.088034864031947e-13...",...,7.042918e-13,5.722276e-13,6.223233e-13,5.703442e-13,3.570969e-13,4.162824e-13,4.730358e-13,3.096919e-13,4.176669e-13,3.454101e-13
2,024,Right,03,"[13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20....","[9.511951793724103e-13, 7.529656792317256e-13,...","[8.999942123011992e-13, 7.426553074306858e-13,...","[1.6925390976709618e-12, 1.0601763124280895e-1...","[7.844179708194547e-13, 5.957220911469509e-13,...","[6.283115748756354e-13, 6.552149821402667e-13,...","[1.0470446388290749e-12, 7.480233255728902e-13...",...,6.305493e-13,5.153313e-13,5.580002e-13,5.031370e-13,3.149557e-13,3.711278e-13,4.069026e-13,2.667660e-13,3.802697e-13,2.877744e-13
3,024,Right,12,"[13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20....","[9.441147341234419e-14, 7.274418241094575e-14,...","[9.121987733213792e-14, 8.119072538923668e-14,...","[1.267965799606627e-13, 8.919558150812431e-14,...","[7.063571409934557e-14, 5.611350518905433e-14,...","[8.909305949369696e-14, 5.817558001771653e-14,...","[9.618901671729255e-14, 6.49955013827536e-14, ...",...,5.152335e-14,4.326514e-14,4.518448e-14,4.460339e-14,2.015693e-14,2.128565e-14,3.032828e-14,1.755182e-14,2.147577e-14,2.214879e-14
4,024,Right,13,"[13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20....","[1.4216056694099896e-13, 1.1115620195516617e-1...","[9.671092120251205e-14, 1.0093164860146917e-13...","[1.829484338946015e-13, 1.2513300293004138e-13...","[8.887085101658428e-14, 8.8573188110667e-14, 9...","[1.5964162842363502e-13, 1.0326965008732892e-1...","[1.397320875890213e-13, 1.1576942786223027e-13...",...,8.291981e-14,6.933578e-14,7.448761e-14,6.836462e-14,3.173297e-14,2.538796e-14,4.437440e-14,2.578828e-14,4.175718e-14,3.186879e-14


### Maximal Power Channel: Beta Analysis of 5x20 sec recordings

Get maximal power channels of 2min recordings only

In [101]:
maximal_power = externalized_short_chunks.rank_power_2min(filtered="notch_and_band_pass_filtered", freq_band="beta", channel_group="ring")


20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist


In [102]:
maximal_power["maximal_power_patient_ids"]

array(['024_Right_01', '024_Left_12', '025_Right_01', '025_Left_12',
       '028_Right_23', '028_Left_12', '029_Right_01', '029_Left_01',
       '030_Right_12', '030_Left_23', '032_Right_12', '032_Left_12',
       '047_Right_12', '047_Left_01', '048_Right_01', '048_Left_01',
       '049_Right_01', '049_Left_01', '052_Right_12', '052_Left_01',
       '056_Right_12', '056_Left_23', '059_Right_23', '059_Left_01',
       '061_Right_01', '061_Left_01', '064_Right_01', '064_Left_23',
       '067_Right_01', '067_Left_01', '069_Right_12', '069_Left_12',
       '071_Right_23', '071_Left_23', '072_Right_12', '072_Left_12',
       '075_Right_01', '075_Left_12', '077_Right_23', '077_Left_01',
       '079_Right_23', '079_Left_01', '080_Right_01', '080_Left_01'],
      dtype=object)

In [75]:
maximal_power["maximal_power_data"].head()

,subject,hemisphere,channel,frequencies,power_spectrum_2min,power_spectrum_20sec_1,power_spectrum_20sec_2,power_spectrum_20sec_3,power_spectrum_20sec_4,power_spectrum_20sec_5,...,power_spectrum_20sec_3_mean,power_spectrum_20sec_4_mean,power_spectrum_20sec_5_mean,power_spectrum_2min_sd,power_spectrum_20sec_1_sd,power_spectrum_20sec_2_sd,power_spectrum_20sec_3_sd,power_spectrum_20sec_4_sd,power_spectrum_20sec_5_sd,rank_2min
0,024,Right,02,"[13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20....","[1.160899811117978e-12, 8.567658832017647e-13,...","[1.112888572779954e-12, 8.456427848714715e-13,...","[1.9864805111607753e-12, 1.22709808887544e-12,...","[1.0017094867206536e-12, 7.040899407980877e-13...","[8.091607529056341e-13, 7.076689567011787e-13,...","[1.3069238039228217e-12, 9.088034864031947e-13...",...,5.722276e-13,6.223233e-13,5.703442e-13,3.570969e-13,4.162824e-13,4.730358e-13,3.096919e-13,4.176669e-13,3.454101e-13,1.0
1,024,Left,03,"[13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20....","[5.381283201138353e-13, 4.4845322232500177e-13...","[6.155536470050576e-13, 4.861205735841604e-13,...","[5.411127621263794e-13, 4.4683978965192593e-13...","[4.185562521321441e-13, 2.658972049450222e-13,...","[3.739025670598156e-13, 3.9286957331246923e-13...","[7.228738597857049e-13, 5.714120896178595e-13,...",...,2.297123e-13,2.499359e-13,2.826872e-13,1.405996e-13,1.720354e-13,1.564652e-13,1.192147e-13,1.344675e-13,1.611019e-13,1.0
2,025,Right,13,"[13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20....","[1.9338718570832658e-13, 1.1204239755129303e-1...","[8.307406335962995e-14, 5.361018974592886e-14,...","[1.071888254866544e-13, 9.201002282365173e-14,...","[4.25060057959006e-13, 2.931071757711625e-13, ...","[1.7519179743949128e-13, 1.1118885463303747e-1...","[8.536656707015274e-14, 4.613287852322604e-14,...",...,9.296208e-14,6.058585e-14,3.340628e-14,3.668433e-14,1.576952e-14,2.326642e-14,9.043813e-14,4.038791e-14,1.604522e-14,1.0
3,025,Left,13,"[13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20....","[1.8644705963285258e-13, 1.6840078497358396e-1...","[1.0552686774924972e-13, 1.2472378058887606e-1...","[1.2552106747880542e-13, 1.0130235447016307e-1...","[2.369140021272072e-13, 1.6269433062857124e-13...","[1.1047893045926817e-13, 1.0487039100714059e-1...","[7.827038598404518e-14, 1.0189844099071544e-13...",...,2.610350e-13,1.548677e-13,7.893212e-14,1.049233e-13,1.185266e-13,1.235302e-13,1.984405e-13,9.120657e-14,3.721684e-14,1.0
4,028,Right,03,"[13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20....","[1.4927599713170888e-13, 9.448328826074599e-14...","[1.2414562638798653e-13, 7.564551295052132e-14...","[1.7901204846946266e-13, 8.185186239607205e-14...","[2.196867161058998e-13, 1.080209559261161e-13,...","[1.52045495234671e-13, 1.3610129149306704e-13,...","[1.8171925290118845e-13, 8.95210496374867e-14,...",...,5.370872e-14,5.112470e-14,4.896996e-14,2.995043e-14,2.537037e-14,3.605375e-14,4.445077e-14,3.493247e-14,3.468680e-14,1.0


In [77]:
maximal_power["ranked_power_2min"].head()

,subject,hemisphere,channel,frequencies,power_spectrum_2min,power_spectrum_20sec_1,power_spectrum_20sec_2,power_spectrum_20sec_3,power_spectrum_20sec_4,power_spectrum_20sec_5,...,power_spectrum_20sec_3_mean,power_spectrum_20sec_4_mean,power_spectrum_20sec_5_mean,power_spectrum_2min_sd,power_spectrum_20sec_1_sd,power_spectrum_20sec_2_sd,power_spectrum_20sec_3_sd,power_spectrum_20sec_4_sd,power_spectrum_20sec_5_sd,rank_2min
0,024,Right,01,"[13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20....","[1.006959873681143e-12, 7.627618606373016e-13,...","[9.227261489012247e-13, 6.856116072561524e-13,...","[1.6238086629250652e-12, 1.0183756423828129e-1...","[7.63183402082647e-13, 6.122674865474809e-13, ...","[7.333007422693277e-13, 6.482917709124258e-13,...","[1.0377924563977723e-12, 8.049693773248429e-13...",...,5.230051e-13,6.110900e-13,5.399850e-13,3.376479e-13,4.029409e-13,4.310370e-13,2.884047e-13,4.183058e-13,3.434652e-13,2.0
1,024,Right,02,"[13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20....","[1.160899811117978e-12, 8.567658832017647e-13,...","[1.112888572779954e-12, 8.456427848714715e-13,...","[1.9864805111607753e-12, 1.22709808887544e-12,...","[1.0017094867206536e-12, 7.040899407980877e-13...","[8.091607529056341e-13, 7.076689567011787e-13,...","[1.3069238039228217e-12, 9.088034864031947e-13...",...,5.722276e-13,6.223233e-13,5.703442e-13,3.570969e-13,4.162824e-13,4.730358e-13,3.096919e-13,4.176669e-13,3.454101e-13,1.0
2,024,Right,03,"[13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20....","[9.511951793724103e-13, 7.529656792317256e-13,...","[8.999942123011992e-13, 7.426553074306858e-13,...","[1.6925390976709618e-12, 1.0601763124280895e-1...","[7.844179708194547e-13, 5.957220911469509e-13,...","[6.283115748756354e-13, 6.552149821402667e-13,...","[1.0470446388290749e-12, 7.480233255728902e-13...",...,5.153313e-13,5.580002e-13,5.031370e-13,3.149557e-13,3.711278e-13,4.069026e-13,2.667660e-13,3.802697e-13,2.877744e-13,3.0
3,024,Right,12,"[13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20....","[9.441147341234419e-14, 7.274418241094575e-14,...","[9.121987733213792e-14, 8.119072538923668e-14,...","[1.267965799606627e-13, 8.919558150812431e-14,...","[7.063571409934557e-14, 5.611350518905433e-14,...","[8.909305949369696e-14, 5.817558001771653e-14,...","[9.618901671729255e-14, 6.49955013827536e-14, ...",...,4.326514e-14,4.518448e-14,4.460339e-14,2.015693e-14,2.128565e-14,3.032828e-14,1.755182e-14,2.147577e-14,2.214879e-14,5.0
4,024,Right,13,"[13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20....","[1.4216056694099896e-13, 1.1115620195516617e-1...","[9.671092120251205e-14, 1.0093164860146917e-13...","[1.829484338946015e-13, 1.2513300293004138e-13...","[8.887085101658428e-14, 8.8573188110667e-14, 9...","[1.5964162842363502e-13, 1.0326965008732892e-1...","[1.397320875890213e-13, 1.1576942786223027e-13...",...,6.933578e-14,7.448761e-14,6.836462e-14,3.173297e-14,2.538796e-14,4.437440e-14,2.578828e-14,4.175718e-14,3.186879e-14,4.0


Shapiro Wilk Test: are the multiple power mean values normally distributed?

In [103]:
shapiro_wilk = externalized_short_chunks.shapiro_wilk_means_distribution(filtered="notch_and_band_pass_filtered", freq_band="beta", channel_group="ring")


20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
20 sec rec number 6 does not exist
Data for patient 024_Right_01, power_spectrum_20sec_5_mean is normally distributed (p=0.5143493413925171)
Data for patient 024_Left_12, power_spectrum_20sec_5_mean is not normally distributed (p=0.019362498074769974)
Data for patient 025_Right_01, power_spectrum_20sec_5_mean is normally distributed (p=0.40928125381469727)
Data for patient 025_Left_12, power_spectrum_20sec_5_mean is normally distributed (p=0.902966320514679)
Data for patient 028_Right_23, power_spectrum_20sec_5_mean is normally distributed (p=0.6732825040817261)
Data for patient 028_Left_12, power_spectrum_2

In [104]:
shapiro_wilk

,patient_id,shapiro_wilk_stat,shapiro_wilk_p,normal_distribution
0,024_Right_01,0.917564,0.514349,True
1,024_Left_12,0.730319,0.019362,False
2,025_Right_01,0.899891,0.409281,True
3,025_Left_12,0.974450,0.902966,True
4,028_Right_23,0.941037,0.673283,True
5,028_Left_12,0.907904,0.455095,True
6,029_Right_01,0.716835,0.014248,False
7,029_Left_01,0.583957,0.000379,False
8,030_Right_12,0.957205,0.788399,True
9,030_Left_23,0.872398,0.276269,True


### Spatial Distribution: Beta rank analysis of 5x20 sec externalized BSSU recordings

Get beta ranks for each bipolar BSSU channel for each chunk

Per subject_hemisphere: 
- get beta average ± SD of all chunks
- choose one channel with maximal beta average (rings)


DF structure: pd.DataFrame({"sub_hemisphere_max_channel": range(1,22),
"chunk_1": [...], "chunk_2": [...], etc })